# 🔧 Fix Migration Links & Inconsistencies

**Purpose**: Fix các vấn đề phát hiện sau migration:
1. **bidding_untitled** (767 chunks) - Old format cần update
2. **FORM-Bidding/2025#bee720** (3 chunks) - Missing trong documents table
3. **4 exam documents** - Có trong documents table nhưng 0 chunks

**Mode**: Run từng cell để debug, có thể rollback nếu sai

## 📦 Setup

In [12]:
import sys
import os
from pathlib import Path

# Add project root
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Project: {project_root}")

📁 Project: /home/sakana/Code/RAG-bidding


In [13]:
import psycopg2
import json
import pandas as pd
from datetime import datetime
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Database config
DB_CONFIG = {
    'host': 'localhost',
    'database': 'rag_bidding_v2',
    'user': 'sakana',
    'password': 'sakana123'
}

print("✅ Imports successful")

✅ Imports successful


In [14]:
# Helper functions

def get_connection():
    """Get database connection."""
    return psycopg2.connect(**DB_CONFIG)

def run_query(query: str, params: tuple = None) -> pd.DataFrame:
    """Run query and return DataFrame."""
    conn = get_connection()
    try:
        df = pd.read_sql_query(query, conn, params=params)
        return df
    finally:
        conn.close()

def execute_query(query: str, params: tuple = None) -> int:
    """Execute query and return affected rows."""
    conn = get_connection()
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        affected = cursor.rowcount
        conn.commit()
        return affected
    except Exception as e:
        conn.rollback()
        raise e
    finally:
        conn.close()

def print_section(title: str):
    """Print formatted section header."""
    print("\n" + "="*80)
    print(f"📊 {title}")
    print("="*80 + "\n")

def determine_category(doc_type: str) -> str:
    """Determine document category from type."""
    mapping = {
        'law': 'Luật chính',
        'decree': 'Nghị định',
        'circular': 'Thông tư',
        'decision': 'Quyết định',
        'bidding': 'Hồ sơ mời thầu',
        'template': 'Mẫu báo cáo',
        'exam': 'Câu hỏi thi'
    }
    return mapping.get(doc_type, 'Khác')

print("✅ Helper functions loaded")

✅ Helper functions loaded


---

## 🔍 Issue 1: Analyze Missing Documents

Documents có trong vector DB nhưng KHÔNG có trong documents table

In [15]:
print_section("Missing Documents Analysis")

query = """
WITH vector_docs AS (
    SELECT 
        cmetadata->>'document_id' as document_id,
        cmetadata->>'document_type' as document_type,
        cmetadata->>'title' as title,
        cmetadata->>'source_file' as source_file,
        COUNT(*) as total_chunks
    FROM langchain_pg_embedding
    GROUP BY 
        cmetadata->>'document_id',
        cmetadata->>'document_type',
        cmetadata->>'title',
        cmetadata->>'source_file'
)
SELECT v.*
FROM vector_docs v
LEFT JOIN documents d ON v.document_id = d.document_id
WHERE d.document_id IS NULL
ORDER BY total_chunks DESC;
"""

missing_docs = run_query(query)

if missing_docs.empty:
    print("✅ All documents in vector DB exist in documents table")
else:
    print(f"⚠️ Found {len(missing_docs)} documents in vector DB but NOT in documents table:\n")
    
    for idx, row in missing_docs.iterrows():
        print(f"[{idx+1}] {row['document_id']}")
        print(f"    Type: {row['document_type'] or 'unknown'}")
        print(f"    Chunks: {row['total_chunks']}")
        print(f"    Title: {row['title'] or 'N/A'}")
        print(f"    Source: {Path(row['source_file']).name if row['source_file'] else 'N/A'}")
        print()

# Store for later use
print(f"\n💾 Stored in variable: missing_docs (DataFrame with {len(missing_docs)} rows)")


📊 Missing Documents Analysis

✅ All documents in vector DB exist in documents table

💾 Stored in variable: missing_docs (DataFrame with 0 rows)


### 1.1: Inspect bidding_untitled Details

Xem chi tiết để quyết định xử lý như thế nào

In [16]:
if not missing_docs.empty and 'bidding_untitled' in missing_docs['document_id'].values:
    print_section("bidding_untitled Detailed Analysis")
    
    # Get sample chunks
    query = """
    SELECT 
        cmetadata->>'title' as title,
        cmetadata->>'source_file' as source_file,
        cmetadata->>'document_type' as doc_type,
        cmetadata->>'chunk_id' as chunk_id,
        cmetadata->>'chunk_index' as chunk_index,
        LEFT(document, 150) as content_preview
    FROM langchain_pg_embedding
    WHERE cmetadata->>'document_id' = 'bidding_untitled'
    ORDER BY (cmetadata->>'chunk_index')::int
    LIMIT 10;
    """
    
    samples = run_query(query)
    
    print(f"📋 Sample chunks (showing 10 of 767):\n")
    
    for idx, row in samples.iterrows():
        print(f"Chunk {idx+1}:")
        print(f"   chunk_id: {row['chunk_id']}")
        print(f"   chunk_index: {row['chunk_index']}")
        print(f"   type: {row['doc_type']}")
        print(f"   title: {row['title']}")
        print(f"   source: {Path(row['source_file']).name if row['source_file'] else 'N/A'}")
        print(f"   content: {row['content_preview']}...")
        print()
    
    print("\n💡 Decision Options:")
    print("   A. Update to proper FORM-* format (recommend: FORM-Bidding-HSMT)")
    print("   B. Delete if this is duplicate/invalid data")
    print("   C. Leave as-is and only backfill documents table")
else:
    print("ℹ️ bidding_untitled not found in missing documents")

ℹ️ bidding_untitled not found in missing documents


### 1.2: Backfill Missing Documents

Insert missing documents vào documents table

In [17]:
print_section("Backfilling Missing Documents")

if missing_docs.empty:
    print("✅ No missing documents to backfill")
else:
    conn = get_connection()
    cursor = conn.cursor()
    
    insert_query = """
    INSERT INTO documents (
        document_id,
        document_name,
        document_type,
        category,
        file_name,
        source_file,
        total_chunks,
        status,
        created_at,
        updated_at
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, NOW(), NOW())
    ON CONFLICT (document_id) DO UPDATE SET
        total_chunks = EXCLUDED.total_chunks,
        updated_at = NOW();
    """
    
    try:
        inserted = 0
        for idx, row in missing_docs.iterrows():
            doc_id = row['document_id']
            doc_type = row['document_type'] or 'unknown'
            title = row['title'] or doc_id
            source_file = row['source_file']
            chunks = row['total_chunks']
            category = determine_category(doc_type)
            
            # Extract file_name from source_file
            if source_file and source_file.strip():
                file_name = Path(source_file).name
                source_path = source_file
            else:
                # Fallback: use document_id as file_name
                file_name = f"{doc_id}.docx"
                source_path = f"/unknown/{file_name}"  # Placeholder path
            
            cursor.execute(insert_query, (
                doc_id,
                title[:200],  # Truncate if too long
                doc_type,
                category,
                file_name,
                source_path,
                chunks,
                'active'
            ))
            inserted += cursor.rowcount
            print(f"✅ Inserted: {doc_id}")
            print(f"   - File: {file_name}")
            print(f"   - Source: {source_path}")
            print(f"   - Chunks: {chunks}")
        
        conn.commit()
        print(f"\n✅ Successfully backfilled {inserted} documents")
        
    except Exception as e:
        conn.rollback()
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        conn.close()


📊 Backfilling Missing Documents

✅ No missing documents to backfill


---

## 🔍 Issue 2: Zero-Chunk Documents

Documents có trong documents table nhưng không có chunks

In [18]:
print_section("Zero-Chunk Documents Analysis")

query = """
SELECT 
    document_id,
    document_name,
    document_type,
    category,
    status,
    source_file,
    created_at
FROM documents
WHERE total_chunks = 0
ORDER BY document_id;
"""

zero_chunk_docs = run_query(query)

if zero_chunk_docs.empty:
    print("✅ All documents have chunks")
else:
    print(f"⚠️ Found {len(zero_chunk_docs)} documents with 0 chunks:\n")
    
    for idx, row in zero_chunk_docs.iterrows():
        print(f"[{idx+1}] {row['document_id']}")
        print(f"    Name: {row['document_name']}")
        print(f"    Type: {row['document_type']} | Category: {row['category']}")
        print(f"    Status: {row['status']}")
        print(f"    Source: {row['source_file'] or 'N/A'}")
        print()

print(f"\n💾 Stored in variable: zero_chunk_docs (DataFrame with {len(zero_chunk_docs)} rows)")
print("\n💡 Options: Mark as 'pending' OR Delete")


📊 Zero-Chunk Documents Analysis

⚠️ Found 4 documents with 0 chunks:

[1] EXAM-Ngân-hàng-câu-hỏi-CC
    Name: Ngân hàng câu hỏi CCDT đợt 2
    Type: exam_question | Category: Câu hỏi thi
    Status: pending
    Source: data/raw/Cau hoi thi/Ngân hàng câu hỏi CCDT đợt 2.pdf

[2] EXAM-Ngân-hàng-câu-hỏi-th
    Name: Ngân hàng câu hỏi thi CCDT đợt 1
    Type: exam_question | Category: Câu hỏi thi
    Status: pending
    Source: data/raw/Cau hoi thi/Ngân hàng câu hỏi thi CCDT đợt 1.pdf

[3] EXAM-NHCH_2692025_dot-2
    Name: NHCH 26.9.2025 dot 2- bổ sung
    Type: exam_question | Category: Câu hỏi thi
    Status: pending
    Source: data/raw/Cau hoi thi/NHCH_26.9.2025_dot 2- bổ sung.pdf

[4] EXAM-NHCH_30925_bo_sung
    Name: NHCH 30.9.25 bo sung theo TB1952 qldt (1)
    Type: exam_question | Category: Câu hỏi thi
    Status: pending
    Source: data/raw/Cau hoi thi/NHCH_30.9.25_bo_sung_theo_TB1952_qldt (1).pdf


💾 Stored in variable: zero_chunk_docs (DataFrame with 4 rows)

💡 Options: Mark a

### 2.1: Mark Zero-Chunk Documents as Pending

In [19]:
# Uncomment to execute:

print_section("Marking Zero-Chunk Documents as Pending")

if zero_chunk_docs.empty:
    print("ℹ️ No documents to mark")
else:
    query = """
    UPDATE documents
    SET status = 'pending', updated_at = NOW()
    WHERE total_chunks = 0;
    """
    
    try:
        affected = execute_query(query)
        print(f"✅ Marked {affected} documents as 'pending'")
    except Exception as e:
        print(f"❌ Error: {e}")

print("ℹ️ Uncomment code above to mark documents as pending")


📊 Marking Zero-Chunk Documents as Pending

✅ Marked 4 documents as 'pending'
ℹ️ Uncomment code above to mark documents as pending


---

## ✅ Final Verification

In [21]:
print_section("Final Consistency Check")

query = """
SELECT 
    (SELECT COUNT(DISTINCT cmetadata->>'document_id') FROM langchain_pg_embedding) as vector_db_docs,
    (SELECT COUNT(*) FROM documents) as documents_table_docs,
    (SELECT COUNT(*) FROM documents WHERE total_chunks = 0) as zero_chunk_docs,
    (SELECT COUNT(*) FROM langchain_pg_embedding WHERE cmetadata->>'document_id' LIKE '%untitled%') as old_format_chunks,
    (SELECT COUNT(*) FROM langchain_pg_embedding WHERE cmetadata->>'document_id' ~ '^(LUA|ND|TT|QD|FORM|TEMPLATE|EXAM)-') as new_format_chunks,
    (SELECT COUNT(*) FROM langchain_pg_embedding) as total_chunks;
"""

stats = run_query(query)
s = stats.iloc[0]

print(f"""📊 Final State:

Vector DB:
   - Unique documents: {s['vector_db_docs']}
   - Total chunks: {s['total_chunks']}
   - New format: {s['new_format_chunks']} ({s['new_format_chunks']/s['total_chunks']*100:.1f}%)
   - Old format: {s['old_format_chunks']} ({s['old_format_chunks']/s['total_chunks']*100:.1f}%)

Documents Table:
   - Total: {s['documents_table_docs']}
   - With chunks: {s['documents_table_docs'] - s['zero_chunk_docs']}
   - Zero chunks: {s['zero_chunk_docs']}

Consistency: {'✅ GOOD' if s['vector_db_docs'] == (s['documents_table_docs'] - s['zero_chunk_docs']) else '⚠️ NEEDS ATTENTION'}
""")

if s['old_format_chunks'] > 0:
    print(f"\n⚠️ Still have {s['old_format_chunks']} old format chunks!")
    print("   These are the 'bidding_untitled' chunks that need manual decision.")


📊 Final Consistency Check

📊 Final State:

Vector DB:
   - Unique documents: 57
   - Total chunks: 6242
   - New format: 5475 (87.7%)
   - Old format: 767 (12.3%)

Documents Table:
   - Total: 61
   - With chunks: 57
   - Zero chunks: 4

Consistency: ✅ GOOD


⚠️ Still have 767 old format chunks!
   These are the 'bidding_untitled' chunks that need manual decision.
